# Lecture 2 - Kinematics

[Lecture 2](https://indico.ific.uv.es/event/6803/contributions/21220) by Vincent Mathieu contains a few data files containing four-momenta data samples. Our goal in this notebook is to identify which reaction was used to generate these data samples.

## `Three-particles-1.dat`

In [ ]:
import numpy as np

filename = "data/Three-particles-1.dat"
data = np.loadtxt(filename)
data.shape

In [ ]:
n_final_state = 3
pa, p1, p2, p3 = (data[i::4].T for i in range(n_final_state + 1))

In [ ]:
def mass(p: np.ndarray) -> np.ndarray:
    return np.sqrt(mass_squared(p))


def mass_squared(p: np.ndarray) -> np.ndarray:
    return p[0] ** 2 - np.sum(p[1:] ** 2, axis=0)


p0 = p1 + p2 + p3
m0 = mass(p0)
print(f"{m0.mean():.4g} +/- {m0.std():.4g}")

In [ ]:
from IPython.display import Math

for i, p in enumerate([p0, p1, p2, p3]):
    display(Math(Rf"m_{i} = {mass(p).mean():.3g}\text{{ GeV}}"))

display(Math(Rf"m_{{a}} = {mass(pa).mean():.3g}\text{{ GeV}}"))

In [ ]:
from particle import Particle


def find_candidates(m: float, delta: float = 0.001) -> list[Particle]:
    return Particle.findall(lambda p: (m - delta) < 1e-3 * p.mass < (m + delta))


m1 = mass(p1).mean()
m2 = mass(p2).mean()
m3 = mass(p3).mean()
particles = tuple(find_candidates(m.mean())[0] for m in [m1, m2, m3])

src = R"\text{Final state: }" + ", ".join(f"{p.latex_name}" for p in particles)
Math(src)

So this is probably some photon&nbsp;$\gamma$ hitting a target that produces a meson&nbsp;$\eta$, pion&nbsp;$\pi^0$, and proton&nbsp;$p$.

### Dalitz plot

By plotting the three Mandelstam variables in a Dalitz plot, we can identify resonances appear in the reaction for which this data was generated.

In [ ]:
s12 = mass_squared(p1 + p2)
s23 = mass_squared(p2 + p3)
s31 = mass_squared(p3 + p1)

m12 = mass(p1 + p2)
m23 = mass(p2 + p3)
m31 = mass(p3 + p1)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
fig.suptitle("Dalitz plot – 2D histogram")
ax.hist2d(s12, s23, bins=100, cmin=1)
ax.set_xlabel(R"$s_{12}\;\left[\mathrm{GeV}^2\right]$")
ax.set_ylabel(R"$s_{23}\;\left[\mathrm{GeV}^2\right]$")
fig.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(10, 4), ncols=2)
fig.suptitle("Dalitz plot – scatter plot")
ax1.scatter(s12, s23, c="black", s=1e-3)
ax1.set_xlabel(R"$s_{12}\;\left[\mathrm{GeV}^2\right]$")
ax1.set_ylabel(R"$s_{23}\;\left[\mathrm{GeV}^2\right]$")
ax2.scatter(s31, s12, c="black", s=1e-3)
ax2.set_xlabel(R"$s_{31}\;\left[\mathrm{GeV}^2\right]$")
ax2.set_ylabel(R"$s_{12}\;\left[\mathrm{GeV}^2\right]$")
fig.tight_layout()
plt.show()

### Particle identification

In the following, we make a few cuts on Mandelstam variables to select a region where the resonances lie isolated. We then use these cuts as a filter on the computed masses for each each and then compute the mean.

In [ ]:
m12_mean = m12[(s12 < 3) & (2.5 < s23) & (s23 < 10)].mean()
m12_mean**2

In [ ]:
m23_mean = m23[s23 < 2.5].mean()
m23_mean**2

In [ ]:
m31_mean = m31[(s12 > 3) & (s31 < 4)].mean()
m31_mean**2

The particle candidates for $R_{12}$, $R_{23}$, and $R_{31}$ are then:

In [ ]:
find_candidates(m12_mean, delta=0.01)

In [ ]:
find_candidates(m23_mean, delta=0.015)

In [ ]:
find_candidates(m31_mean, delta=0.01)